In [1]:
%matplotlib inline

In [2]:
import numpy as np
import xarray as xr
import pandas as pd

import scipy.spatial
import scipy.linalg

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.sandbox.stats.multicomp as smm
import statsmodels.tsa.stattools as tsa
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import matplotlib.pyplot as plt
import seaborn as sns
#current_palette = sns.color_palette("hls", 8)
#sns.set_palette(current_palette)

In [ ]:
# Load the data
# wget ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G10010/G10010_SIBT1850_v1.1.zip
nc = xr.open_dataset('data/G10010_SIBT1850_v1.1.nc')

In [ ]:
# Average out years
yr_avg = nc.seaice_conc.groupby('time.year').mean()

# Separate into seasons
seasons = nc.seaice_conc.groupby('time.season').mean('latitude').mean('longitude')
seas_df = seasons.to_dataframe()
seas_df = seas_df.groupby([lambda x: x.year, 'season']).mean()
seas_df = seas_df.reset_index()

djf_df = seas_df.loc[seas_df['season'] == 'DJF']
jja_df = seas_df.loc[seas_df['season'] == 'JJA']
mam_df = seas_df.loc[seas_df['season'] == 'MAM']
son_df = seas_df.loc[seas_df['season'] == 'SON']

djf_df.reset_index(inplace=True)
jja_df.reset_index(inplace=True)
mam_df.reset_index(inplace=True)
son_df.reset_index(inplace=True)

del djf_df['index']
del jja_df['index']
del mam_df['index']
del son_df['index']

djf_df.columns = ['year', 'season', 'seaice_conc']
jja_df.columns = ['year', 'season', 'seaice_conc']
mam_df.columns = ['year', 'season', 'seaice_conc']
son_df.columns = ['year', 'season', 'seaice_conc']

del djf_df['season']
del jja_df['season']
del mam_df['season']
del son_df['season']

In [ ]:
seasons = ['JJA', 'SON', 'DJF', 'MAM']
dfs = {'DJF': djf_df, 'JJA': jja_df, 'MAM': mam_df, 'SON': son_df}

In [ ]:
cmap = {'JJA': 'crimson', 'SON': 'darkorange', 'DJF': 'steelblue', 'MAM': 'seagreen'}

## Arctic splines

Since the arctic sea data set is such a long time series, we'll compute linear splines to get a piecewise trend of the data.

#### Definitions:

* A **linear spline** is a continuous function formed by connecting linear
segments.  The points where the segments connect are called the
**knots** of the spline.

* A **spline of degree** $D$ is a function formed by connecting polynomial segments
of degree $D$ so that:
    * the function is continuous,
    * the function has $D-1$ continuous derivatives,
    * the $D^{\text{th}}$ derivative is constant between knots

* The **truncated polynomial** of degree $D$ associated with knot $\xi_{k}$ is the function which is equal to $0$ to the left of $\xi_{k}$ and equal to $(x-\xi_{k})^D$ to the right of $\xi_{k}$.

$$(x-\xi_{k})_{+}^D = 
\begin{cases}
(x-\xi_{k})^D, & \text{if $x \geq \xi_{k}$} \\
0, & \text{if $x \lt \xi_{k}$}
\end{cases}
$$


#### Equation of spline of degree $D$ and $K$ knots:

$$y = \beta_{0} + \sum_{d=1}^{D}\beta_{d}x_{d} + \sum_{k=1}^{K}\alpha_{k}(x-\xi_{k})_{+}^D$$

#### Design Matrix:
$$
\mathbf{X} = 
\begin{bmatrix}
1 & x_1 & x_{1}^{2} & \dots & x_{1}^{D} & (x_{1}-\xi_{1})_{+}^D & \dots & (x_{1}-\xi_{K})_{+}^D \\
1 & x_2 & x_{2}^{2} & \dots & x_{2}^{D} & (x_{2}-\xi_{1})_{+}^D & \dots & (x_{2}-\xi_{K})_{+}^D \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
1 & x_n & x_{n}^{2} &\dots & x_{n}^{D} & (x_{n}-\xi_{1})_{+}^D & \dots & (x_{n}-\xi_{K})_{+}^D
\end{bmatrix}
$$

In [ ]:
def linear_splines(data, knots, degree=1):
    minyr = data.year.min()
    nyrs = data.year.shape[0]

    # Use indices of years for knots
    minyr = data.year.min()
    knots2 = [k-minyr for k in knots]
    
    # Column vector for beta1 are (t1, t2,...,tn)
    X1 = np.arange(nyrs)
    X1 = X1[:, np.newaxis]
    
    # Generate design matrix
    X2 = np.zeros((nyrs, len(knots2)))
    for col, k in enumerate(knots2):
        X2[k:, col] = np.abs(X2[k:, col] - X1[:nyrs-k, 0])
    X = np.hstack((X1, X2))
    return smf.glm('seaice_conc ~ X', data=data).fit()

### Knots for Arctic sea

From the explore_seaice notebook, the trends look peicewise with possible knots at various locations. We'll define our set of knots for each season:

$$
K_{\text{JJA}} = \{1917, 1944, 1996\}\\
K_{\text{SON}} = \{1917, 1943, 1996\}\\
K_{\text{DJF}} = \{1974, 1996\}\\
K_{\text{MAM}} = \{1933, 1979\}
$$

We'll plot the knots to visually check

In [ ]:
# Define the knots
jjaknots = [1917, 1944, 1996]
#jjaknots2 = [1893, 1899, 1917, 1943, 1996]
sonknots = [1917, 1943, 1996]
djfknots = [1974, 1996]
mamknots = [1933, 1979]

splines = {'JJA': linear_splines(jja_df, jjaknots),
           'SON': linear_splines(son_df, sonknots),
           'DJF': linear_splines(djf_df, djfknots),
           'MAM': linear_splines(mam_df, mamknots)}

In [ ]:
def plot_splines(yrs, conc, preds, label, season=None, ax=None, col='g'):
    if ax is None:
        ax = plt.gca()
    spl = ax.plot(yrs, conc, alpha=0.3, color='k')
    ax.plot(yrs, preds, label=label, color=col)
    ax.legend()
    ax.set_xlim(1845, 2016)
    #ax.set_ylim(17.5, 26)
    ax.set_xlabel('years')
    ax.set_ylabel('seaice_conc')
    ax.set_title('Linear Splines for {}'.format(season))
    return spl

In [ ]:
f, axs = plt.subplots(2, 2, figsize=(15,10))
axs = np.array(axs)

for i, ax in enumerate(axs.reshape(-1)):
    s = seasons[i]
    yr = dfs[s].year
    conc = dfs[s].seaice_conc
    preds = splines[s].fittedvalues
    plot_splines(yr, conc, preds, 'splines', season=s, ax=ax, col=cmap[s])

Visually the splines look reasonable. However, let's test the significance of each spline and remove the splines that are not significicant with a significance level, $\alpha = 0.05$

We'll use Bonferroni's correction for multiple tests.

In [ ]:
nyrs = yr_avg.shape[0]
#nparams = len(knots) + 2

# Estimates
mu_hat = {season: spl.mu for season, spl in splines.items()}
R = {season: spl.resid_response.values for season, spl in splines.items()}
cov = {season: spl.cov_params() for season, spl in splines.items()}
#sigma_hat2 = {season: np.sum(r**2) / (nyrs-nparams) for season, r in R.items()}
pvals = {season: spl.pvalues.values for season, spl in splines.items()}

In [ ]:
bonferonnis = {season: (smm.multipletests(pval, method='bonferroni')[:2])
               for season, pval in pvals.items()}

In [ ]:
for season, test in bonferonnis.items():
    print('{}: {}'.format(season, test[0]))

We see some splines are not significant. We'll remove those and recompute our model. Our new knots are now:

$$
K_{\text{JJA}} = \{1944, 1996\}\\
K_{\text{SON}} = \{1917, 1996\}\\
K_{\text{DJF}} = \{1996\}\\
K_{\text{MAM}} = \{1933, 1979\}
$$


In [ ]:
# Splines with new knots... Pt.2
jjaknots2 = [1944, 1996]
sonknots2 = [1917, 1996]
djfknots2 = [1996]
mamknots2 = [1933, 1979]

splines2 = {'JJA': linear_splines(jja_df, jjaknots2),
           'SON': linear_splines(son_df, sonknots2),
           'DJF': linear_splines(djf_df, djfknots2),
           'MAM': linear_splines(mam_df, mamknots2)}

In [ ]:
f, axs = plt.subplots(2, 2, figsize=(15,10))
axs = np.array(axs)

for i, ax in enumerate(axs.reshape(-1)):
    s = seasons[i]
    yr = dfs[s].year
    conc = dfs[s].seaice_conc
    preds = splines2[s].fittedvalues
    plot_splines(yr, conc, preds, 'splines', season=s, ax=ax, col=cmap[s])

In [ ]:
# Estimates Pt.2
mu_hat2 = {season: spl.mu for season, spl in splines2.items()}
R2 = {season: spl.resid_response.values for season, spl in splines2.items()}
cov2 = {season: spl.cov_params() for season, spl in splines2.items()}
pvals2 = {season: spl.pvalues.values for season, spl in splines2.items()}

bonferonnis2 = {season: (smm.multipletests(pval, method='bonferroni')[:2])
               for season, pval in pvals2.items()}

for season, test in bonferonnis2.items():
    print('{}: {}'.format(season, test[0]))

Once again, we see some splines that don't make the cut. We'll remove those, and recompute our model. Our knots are now:

$$
K_{\text{JJA}} = \{1996\}\\
K_{\text{SON}} = \{1996\}\\
K_{\text{DJF}} = \{1996\}\\
K_{\text{MAM}} = \{1933, 1979\}
$$


In [ ]:
# Estimates Pt.3
jjaknots3 = [1996]
sonknots3 = [1996]
djfknots3 = [1996]
mamknots3 = [1933, 1979]

splines3 = {'JJA': linear_splines(jja_df, jjaknots3),
           'SON': linear_splines(son_df, sonknots3),
           'DJF': linear_splines(djf_df, djfknots3),
           'MAM': linear_splines(mam_df, mamknots3)}

f, axs = plt.subplots(2, 2, figsize=(15,10))
axs = np.array(axs)

for i, ax in enumerate(axs.reshape(-1)):
    s = seasons[i]
    yr = dfs[s].year
    conc = dfs[s].seaice_conc
    preds = splines3[s].fittedvalues
    plot_splines(yr, conc, preds, 'splines', season=s, ax=ax, col=cmap[s])

In [ ]:
# Bonferonnis Pt.3
mu_hat3 = {season: spl.mu for season, spl in splines3.items()}
R3 = {season: spl.resid_response.values for season, spl in splines3.items()}
cov3 = {season: spl.cov_params() for season, spl in splines3.items()}
pvals3 = {season: spl.pvalues.values for season, spl in splines3.items()}

bonferonnis3 = {season: (smm.multipletests(pval, method='bonferroni')[:2])
               for season, pval in pvals3.items()}

for season, test in bonferonnis3.items():
    print('{}: {}'.format(season, test[0]))

Finally! We have a set of knots that are significant.

In [ ]:
# Rename R3 to something more meaningful: residuals
residuals = R3

### Plot ACFs and PACFs of the residiuals for each season

In [ ]:
# Plot ACFs and PACFs
f, axs = plt.subplots(4, 2, figsize=(18,16))

jja_acf = plot_acf(residuals['JJA'], lags=100, title ='JJA - ACF', ax=axs[0,0])
jja_pacf = plot_pacf(residuals['JJA'], lags=100, title='JJA - PACF', ax=axs[0,1])

son_acf = plot_acf(residuals['SON'], lags=100, title='SON - ACF', ax=axs[1,0])
son_pacf = plot_pacf(residuals['SON'], lags=100, title='SON - PACF', ax=axs[1,1])

djf_acf = plot_acf(residuals['DJF'], lags=100, title='DJF - ACF', ax=axs[2,0])
djf_pacf = plot_pacf(residuals['DJF'], lags=100, title='DJF - PACF', ax=axs[2,1])

mam_acf = plot_acf(residuals['MAM'], lags=100, title='MAM - ACF', ax=axs[3,0])
mam_pacf = plot_pacf(residuals['MAM'], lags=100, title='MAM - PACF', ax=axs[3,1])

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
f, axr = plt.subplots(4, 2, figsize=(18,16))

axr[0,0].scatter(jja_df.year, residuals['JJA'], color=cmap['JJA'])
axr[0,0].set_ylabel('Residual')
axr[0,0].set_title('JJA')
axr[0,1].scatter(jja_df.seaice_conc, residuals['JJA'], color=cmap['JJA'])

axr[1,0].scatter(son_df.year, residuals['SON'], color=cmap['SON'])
axr[1,0].set_ylabel('Residual')
axr[1,0].set_title('SON')
axr[1,1].scatter(son_df.seaice_conc, residuals['SON'], color=cmap['SON'])

axr[2,0].scatter(djf_df.year, residuals['DJF'], color=cmap['DJF'])
axr[2,0].set_ylabel('Residual')
axr[2,0].set_title('DJF')
axr[2,1].scatter(djf_df.seaice_conc, residuals['DJF'], color=cmap['DJF'])

axr[3,0].scatter(mam_df.year, residuals['MAM'], color=cmap['MAM'])
axr[3,0].set_ylabel('Residual')
axr[3,0].set_title('MAM')
axr[3,1].scatter(mam_df.seaice_conc, residuals['MAM'], color=cmap['MAM'])

axr[3,0].set_xlabel('Years')
axr[3,1].set_xlabel('Seaice Concentration')

In [ ]:
# Compute FFT of the residuals
# Take the periodogram (abs(R)^2)
# Plot the periodogram

In [ ]:
ffts = {season: np.fft.fft(resid) for season, resid in residuals.items()}
periodograms = {season: np.abs(fftr)**2 for season, fftr in ffts.items()}

In [ ]:
f, ax = plt.subplots(2, 2, figsize=(18,16))

ax[0,0].plot(periodograms['JJA'], color=cmap['JJA'])
ax[0,0].set_title('JJA')

ax[0,1].plot(periodograms['SON'], color=cmap['SON'])
ax[0,1].set_title('SON')

ax[1,0].plot(periodograms['DJF'], color=cmap['DJF'])
ax[1,0].set_title('DJF')

ax[1,1].plot(periodograms['MAM'], color=cmap['MAM'])
ax[1,1].set_title('MAM')

In [ ]:
f, ax = plt.subplots(2, 2, figsize=(18,16))

ax[0,0].plot(periodograms['JJA'][:8], color=cmap['JJA'])
ax[0,0].set_title('JJA')

ax[0,1].plot(periodograms['SON'][:8], color=cmap['SON'])
ax[0,1].set_title('SON')

ax[1,0].plot(periodograms['DJF'][:8], color=cmap['DJF'])
ax[1,0].set_title('DJF')

ax[1,1].plot(periodograms['MAM'][5:20], color=cmap['MAM'])
ax[1,1].set_title('MAM')

In [ ]:
t1 = yr_avg.year.values
nyrs = t1.max() - t1.min() + 1

# JJA

In [ ]:
y_jja = jja_df.seaice_conc.values
sin_jja = np.array([np.sin(2*np.pi*2 * (t/nyrs)) for t in np.arange(nyrs)])
cos_jja = np.array([np.cos(2*np.pi*2 * (t/nyrs)) for t in np.arange(nyrs)])

In [ ]:
fit_jja = smf.glm('y_jja ~ sin_jja + cos_jja', data=jja_df).fit()
fit_jja.summary()

In [ ]:
print(fit_jja.params)
k_hat_jja = fit_jja.params.values  # extract fitted regression coefficients
e_jja = fit_jja.resid_response    # extract raw residuals

In [ ]:
# Check to see if the mean of residuals are numerically close to 0
print(np.mean(e_jja))

In [ ]:
plt.scatter(jja_df.year, e_jja)

### Fit seasonal cycle with sine removed

In [ ]:
# fitted seasonal cycle
f_hat_jja = k_hat_jja[0] + k_hat_jja[2]*cos_jja  
plt.plot(f_hat_jja,'k-') 

In [ ]:
# Remove residuals and subtract mean for second order residuals: R^2
resid2_jja = residuals['JJA'] - f_hat_jja
resid2_jja = resid2_jja - np.mean(resid2_jja)

In [ ]:
import statsmodels.tsa.arima_model as arima

# Fit AR2 model for R^2
fit2_jja = arima.ARIMA(resid2_jja, (2,0,0)).fit()
fit2_jja.summary()

In [ ]:
# Remove fitted terms from residual (R^3) to check for white noise
resid3_jja = fit2_jja.resid
phi_hat_jja = fit2_jja.arparams

X1_jja = resid3_jja[1:]
X2_jja = resid3_jja[:-1]

X1_jja = X1_jja[:, np.newaxis]
X2_jja = X2_jja[:, np.newaxis]

X_jja = np.hstack((X1_jja, X2_jja))
E_jja = resid3_jja[1:] - np.dot(X_jja, phi_hat_jja)

In [ ]:
plt.scatter(np.arange(len(E_jja)), E_jja)

In [ ]:
ejja_acf = plot_acf(E_jja)
ejja_pacf = plot_pacf(E_jja)

# SON

In [ ]:
y_son = son_df.seaice_conc.values
sin_son = np.array([np.sin(2*np.pi*3 * (t/nyrs)) for t in np.arange(nyrs)])
cos_son = np.array([np.cos(2*np.pi*3 * (t/nyrs)) for t in np.arange(nyrs)])

In [ ]:
fit_son = smf.glm('y_son ~ sin_son + cos_son', data=son_df).fit()
fit_son.summary()

In [ ]:
# Check pvalues for coefficients
fit_son.pvalues

In [ ]:
print(fit_son.params)
k_hat_son = fit_son.params.values  # extract fitted regression coefficients
e_son = fit_son.resid_response    # extract raw residuals

In [ ]:
plt.scatter(son_df.year, e_son)

In [ ]:
# fitted seasonal cycle
f_hat_son = k_hat_son[0] + k_hat_son[1]*sin_son + k_hat_son[2]*cos_son
plt.plot(f_hat_son,'k-')

In [ ]:
# Remove residuals and subtract mean for second order residuals: R^2
resid2_son = residuals['SON'] - f_hat_son
resid2_son = resid2_son - np.mean(resid2_son)

In [ ]:
# Fit AR2 model for R^2
fit2_son = arima.ARIMA(resid2_son, (2,0,0)).fit()
fit2_son.summary()

In [ ]:
# Remove fitted terms from residual (R^3) to check for white noise
resid3_son = fit2_son.resid
phi_hat_son = fit2_son.arparams

X1_son = resid3_son[1:]
X2_son = resid3_son[:-1]

X1_son = X1_son[:, np.newaxis]
X2_son = X2_son[:, np.newaxis]

X_son = np.hstack((X1_son, X2_son))
E_son = resid3_son[1:] - np.dot(X_son, phi_hat_son)

In [ ]:
plt.scatter(np.arange(len(E_son)), E_son)

In [ ]:
eson_acf = plot_acf(E_son)
eson_pacf = plot_pacf(E_son)

# DJF

In [ ]:
y_djf = djf_df.seaice_conc.values
sin_djf = np.array([np.sin(2*np.pi*4 * (t/nyrs)) for t in np.arange(nyrs)])
cos_djf = np.array([np.cos(2*np.pi*4 * (t/nyrs)) for t in np.arange(nyrs)])

In [ ]:
fit_djf = smf.glm('y_djf ~ sin_djf + cos_djf', data=djf_df).fit()
fit_djf.summary()

In [ ]:
# Check pvalues for coefficients
fit_djf.pvalues

In [ ]:
print(fit_djf.params)
k_hat_djf = fit_djf.params.values  # extract fitted regression coefficients
e_djf = fit_djf.resid_response    # extract raw residuals

In [ ]:
plt.scatter(djf_df.year, e_djf)

In [ ]:
# fitted seasonal cycle
f_hat_djf = k_hat_djf[0] + k_hat_djf[1]*sin_djf + k_hat_djf[2]*cos_djf
plt.plot(f_hat_djf,'k-')

In [ ]:
# Remove residuals and subtract mean for second order residuals: R^2
resid2_djf = residuals['DJF'] - f_hat_djf
resid2_djf = resid2_djf - np.mean(resid2_djf)

In [ ]:
# Fit AR2 model for R^2
fit2_djf = arima.ARIMA(resid2_djf, (2,0,0)).fit()
fit2_djf.summary()

In [ ]:
# Remove fitted terms from residual (R^3) to check for white noise
resid3_djf = fit2_djf.resid
phi_hat_djf = fit2_djf.arparams

X1_djf = resid3_djf[1:]
X2_djf = resid3_djf[:-1]

X1_djf = X1_djf[:, np.newaxis]
X2_djf = X2_djf[:, np.newaxis]

X_djf = np.hstack((X1_djf, X2_djf))
E_djf = resid3_djf[1:] - np.dot(X_djf, phi_hat_djf)

In [ ]:
plt.scatter(np.arange(len(E_djf)), E_djf)

In [ ]:
edjf_acf = plot_acf(E_djf)
edjf_pacf = plot_pacf(E_djf)

# MAM

In [ ]:
y_mam = mam_df.seaice_conc.values
sin_mam = np.array([np.sin(2*np.pi*2 * (t/nyrs)) for t in np.arange(nyrs)])
cos_mam = np.array([np.cos(2*np.pi*2 * (t/nyrs)) for t in np.arange(nyrs)])

In [ ]:
fit_mam = smf.glm('y_mam ~ sin_mam + cos_mam', data=mam_df).fit()
fit_mam.summary()

In [ ]:
# Check pvalues for coefficients
fit_mam.pvalues

In [ ]:
print(fit_mam.params)
k_hat_mam = fit_mam.params.values  # extract fitted regression coefficients
e_mam = fit_mam.resid_response    # extract raw residuals

In [ ]:
plt.scatter(mam_df.year, e_mam)

In [ ]:
# fitted seasonal cycle
f_hat_mam = k_hat_mam[0] + k_hat_mam[1]*sin_mam + k_hat_mam[2]*cos_mam
plt.plot(f_hat_mam,'k-')

In [ ]:
# Remove residuals and subtract mean for second order residuals: R^2
resid2_mam = residuals['MAM'] - f_hat_mam
resid2_mam = resid2_mam - np.mean(resid2_mam)

In [ ]:
# Fit AR2 model for R^2
fit2_mam = arima.ARIMA(resid2_mam, (2,0,0)).fit()
fit2_mam.summary()

In [ ]:
# Remove fitted terms from residual (R^3) to check for white noise
resid3_mam = fit2_mam.resid
phi_hat_mam = fit2_mam.arparams

X1_mam = resid3_mam[1:]
X2_mam = resid3_mam[:-1]

X1_mam = X1_mam[:, np.newaxis]
X2_mam = X2_mam[:, np.newaxis]

X_mam = np.hstack((X1_mam, X2_mam))
E_mam = resid3_mam[1:] - np.dot(X_mam, phi_hat_mam)

In [ ]:
plt.scatter(np.arange(len(E_mam)), E_mam)

In [ ]:
emam_acf = plot_acf(E_mam)
emam_pacf = plot_pacf(E_mam)